In [1]:
import random
random.seed(42)
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

from scipy.stats import invwishart
from numpy import sum, sqrt, outer, transpose, diag
from numpy.random import multivariate_normal
from scipy.sparse import coo_matrix
from numpy.linalg import inv
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE

In [2]:
rating = pd.read_csv("rating.csv")
rating = rating.sample(10000, random_state=42)
rating.describe()

,userId,movieId,rating
count,10000.000000,10000.000000,10000.000000
mean,69946.197800,9397.508700,3.513300
std,40270.913643,20266.782058,1.048705
min,14.000000,1.000000,0.500000
25%,35046.750000,904.000000,3.000000
50%,70725.500000,2250.500000,3.500000
75%,105056.000000,4878.000000,4.000000
max,138456.000000,130512.000000,5.000000


In [3]:
rating_matrix = rating.pivot(index='userId', columns='movieId', values='rating')
rating_matrix = rating_matrix.fillna(0)
R = rating_matrix.to_numpy()

### R = U^T*V   ->   R = U^TBV

In [4]:
# hyperparams: \Theta_0 = {\mu_0, T_0, \nu_0, S_0, sigma}
# \Theta_U = {\mu_U, \Sigma_U}
# \Theta_V = {\mu_V, \Sigma_V}
# \Theta_B = {\mu_B, \Sigma_B}

# Sample \Theta_U, \Theta_V, \Theta_B
# Sample U, V, B
# Compute R

In [5]:
# Initialization
M = rating_matrix.shape[0] # number of customers/users
N = rating_matrix.shape[1] # number of movies
D = 100 # number of latent features
U = np.ones((D, M))
V = np.ones((D, N))
diag_b = range(1, D + 1)
B = diag(diag_b)

mu_0, T_0 = np.zeros((D, 1)), np.eye(D)
nu_0, S_0 = D, np.eye(D)
sigma = 1

In [6]:
nonzero_indices = R.nonzero()
nonzero_values = R[nonzero_indices]

all_indices = list(range(10000))
tr_group = random.sample(all_indices, 8000)
te_group = [i for i in all_indices if i not in tr_group]

tr_indices = (nonzero_indices[0][tr_group], nonzero_indices[1][tr_group])
te_indices = (nonzero_indices[0][te_group], nonzero_indices[1][te_group])

tr_values, te_values = R[tr_indices], R[te_indices]
R_flag = np.zeros_like(R, dtype=int)
R_flag[tr_indices] = 1

In [7]:
iters = 20

Sigma_u, Sigma_v, Sigma_b = np.eye(D), np.eye(D), np.eye(D)

results_df = pd.DataFrame(columns=['Iteration', 'Train_MAE', 'Test_MAE', 'Train_RMSE', 'Test_RMSE'])

for it in range(iters):
    # Sample \Theta_U
    mu_ustar = inv(inv(T_0) + M*inv(Sigma_u))@(inv(T_0)@mu_0 + inv(Sigma_u)@sum(U, axis=1, keepdims=True))
    T_ustar = inv(inv(T_0) + M*inv(Sigma_u))
    mu_u = multivariate_normal(np.squeeze(np.asarray(transpose(mu_ustar))), T_ustar, 1)
    
    nu_ustar = nu_0 + M
    S_ustar = S_0 + (U - transpose(mu_u))@transpose(U - transpose(mu_u))
    inv_wishart = invwishart(nu_ustar, S_ustar)
    Sigma_u = inv_wishart.rvs()
    
    # Sample \Theta_V
    mu_vstar = inv(inv(T_0) + N*inv(Sigma_v))@(inv(T_0)@mu_0 + inv(Sigma_v)@sum(V, axis=1, keepdims=True))
    T_vstar = inv(inv(T_0) + N*inv(Sigma_v))
    mu_v = multivariate_normal(np.squeeze(np.asarray(transpose(mu_vstar))), T_vstar, 1)
    
    nu_vstar = nu_0 + N
    S_vstar = S_0 + (V - transpose(mu_v))@transpose(V - transpose(mu_v))
    inv_wishart = invwishart(nu_vstar, S_vstar)
    Sigma_v = inv_wishart.rvs()
    
    # Sample \Theta_B
    mu_bstar = inv(inv(T_0) + 1*inv(Sigma_b))@(inv(T_0)@mu_0 + inv(Sigma_b)@sum(B, axis=1, keepdims=True))
    T_bstar = inv(inv(T_0) + 1*inv(Sigma_b))
    mu_b = multivariate_normal(np.squeeze(np.asarray(transpose(mu_bstar))), T_bstar, 1)
    
    nu_bstar = nu_0 + 1
    S_bstar = S_0 + outer(diag_b - mu_b, diag_b - mu_b)
    inv_wishart = invwishart(nu_bstar, S_bstar)
    Sigma_b = inv_wishart.rvs()
    
    # Sample U
    weighted_V = B@V
    for i in range(M):
        Lambda_ustar = inv(Sigma_u)
        theta_ustar = transpose(inv(Sigma_u)@transpose(mu_u))
        for j in range(N):
            if R_flag[i, j] == 0:
                continue
            V_j = transpose(weighted_V[:, j])
            Lambda_ustar += outer(V_j, V_j)/sigma
            theta_ustar += V_j*R[i, j]/sigma
        theta_ustar = inv(Lambda_ustar)@transpose(theta_ustar)
        U[:, i] = multivariate_normal(np.squeeze(np.asarray(transpose(theta_ustar))), inv(Lambda_ustar), 1)
        
    # Sample V
    weighted_U = transpose(transpose(U)@B)
    for j in range(N):
        Lambda_vstar = inv(Sigma_v)
        theta_vstar = transpose(inv(Sigma_v)@transpose(mu_v))
        for i in range(M):
            if R[i, j] == 0:
                continue
            U_i = transpose(weighted_U[:, i])
            Lambda_vstar += outer(U_i, U_i)/sigma
            theta_vstar += U_i*R[i, j]/sigma
        theta_vstar = inv(Lambda_vstar)@transpose(theta_vstar)
        V[:, j] = multivariate_normal(np.squeeze(np.asarray(transpose(theta_vstar))), inv(Lambda_vstar), 1)
    
    # Sample B
    Lambda_bstar = inv(Sigma_b)
    theta_bstar = transpose(inv(Sigma_b)@transpose(mu_b))
    for k in range(len(nonzero_indices[0])):
        i = nonzero_indices[0][k]
        j = nonzero_indices[1][k]
        A = U[:, i] * V[:, j]
        Lambda_bstar += outer(A, A)/sigma
        theta_bstar += A*R[i, j]/sigma
    theta_bstar = inv(Lambda_bstar)@transpose(theta_bstar)
    diag_b = multivariate_normal(np.squeeze(np.asarray(transpose(theta_bstar))), inv(Lambda_bstar), 1)[0]
    B = diag(diag_b)
    
    R_star = transpose(U)@B@V
    tr_pred_values = R_star[tr_indices]
    te_pred_values = R_star[te_indices]
    
    tr_mae = MAE(tr_values, tr_pred_values)
    te_mae = MAE(te_values, te_pred_values)
    tr_rmse = np.sqrt(MSE(tr_values, tr_pred_values))
    te_rmse = np.sqrt(MSE(te_values, te_pred_values))
    
    results_df = results_df.append({
        'Iteration': it,
        'Train_MAE': tr_mae,
        'Test_MAE': te_mae,
        'Train_RMSE': tr_rmse,
        'Test_RMSE': te_rmse
    }, ignore_index=True)
    
    if it % 2 == 0:
        print(f"Iteration {it}: Train MAE={tr_mae:.4f}, Test MAE={te_mae:.4f}, Train RMSE={tr_rmse:.4f}, Test RMSE={te_rmse:.4f}")

Iteration 0: Train MAE=0.9018, Test MAE=0.8825, Train RMSE=1.1334, Test RMSE=1.1114
Iteration 2: Train MAE=0.6379, Test MAE=0.6940, Train RMSE=0.8056, Test RMSE=0.8868
Iteration 4: Train MAE=0.6737, Test MAE=0.7341, Train RMSE=0.8484, Test RMSE=0.9137
Iteration 6: Train MAE=0.7114, Test MAE=0.7557, Train RMSE=0.8979, Test RMSE=0.9548
Iteration 8: Train MAE=0.7209, Test MAE=0.7700, Train RMSE=0.9117, Test RMSE=0.9707
Iteration 10: Train MAE=0.7310, Test MAE=0.7679, Train RMSE=0.9243, Test RMSE=0.9699
Iteration 12: Train MAE=0.7364, Test MAE=0.7651, Train RMSE=0.9311, Test RMSE=0.9631
Iteration 14: Train MAE=0.7355, Test MAE=0.7620, Train RMSE=0.9299, Test RMSE=0.9581
Iteration 16: Train MAE=0.7363, Test MAE=0.7687, Train RMSE=0.9321, Test RMSE=0.9608
Iteration 18: Train MAE=0.7467, Test MAE=0.7572, Train RMSE=0.9407, Test RMSE=0.9580


In [8]:
results_df.to_csv(f'full_results_D={D}.csv', index=False)

### k = 1、5、10、20、...